# Introduction to Python, Jupyter, Pandas, and Matplotlib
This is a Jupyter Python notebook, which is a collection of cells. Each cell is either of type 'markdown' (formatted text, like this cell) or code (python, grey background). The two most important rules of Jupyter Notebooks are:
1. ***SHIFT-ENTER*** will cause the current cell to execute. 
  - For Markdown cells, 'execute' means render the formatting. ([Here's a markdown cheatsheet](https://sqlbak.com/blog/wp-content/uploads/2020/04/Jupyter-Notebook-Markdown-Cheatsheet.pdf))
  - For Code cells, 'execute' means run the python.
  - Some Code cells take a while to execute, watch for the * to change to a number
1. Any cell can be edited (double-click into it) and re-executed (SHIFT-ENTER again).
--- 

The first code in any Python script/Jupyter notebook, needs to import any libraries that will be used. The `as` directives allow specification of nicknames that are more convenient to type.

In [ ]:
# This next line is a special command for making plots look better in Jupyter notebooks
%matplotlib inline
import matplotlib.pyplot as plt  # This is for creating graphs
import pandas            as pd   # This greatly simplifies handling of tabular (csv) data

The `pandas` function `read_csv()` can read .csv files on your computer, but it's so smart it can even slurp in a .csv directly from online. This reads in the csv data linked to on this page: https://ourworldindata.org/coronavirus-source-data). 

It takes a few seconds to download, watch for the `*` to change to a number. 

Two two most important terms in the grammar of pandas are `Series` and `DataFrame`. A `DataFrame` is equivalent to  one tab of a spreadsheet (or one worksheet of a workbook). Each column of the `DataFrame` is one `Series`. The data object returned by `read_csv()` is held in a variable named `df` which stands for `DataFrame`.

Note: Python is happy with specifying text constants (strings) with either single or double quotes, but single are preferred because they read cleaner.

In [ ]:
df = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv')

One thing you can do it Jupyter Notebooks that you can't do in regular Python scripts (programs), is to just mention something at the end of a code cell, which is a request for the notebook to display it. If it's too big to be printed completely, it will be automatically summarized. For starters, you can try to get a glimpse of the whole `DataFrame` object itself. The `NaN` ('not a number') are empty cells in the csv.

In [ ]:
df

Here are a few more ways to describe/understand the data, execute each cell and take a minute to read and understand what information is provided:

In [ ]:
df.shape     # note this is a 'data member', a variable that belongs to every data frame. (Rows, Columns)

In [ ]:
df.columns   # pandas assumes the first row of the csv are column headers

In [ ]:
# In Python, anything after a hash is a comment, ignored as code, but for the benefit of the reader

# This is a 'class function', so it needs (); sometimes input parameters go in there.
df.info() # Data size, column headers, counts, and types

In [ ]:
df.describe() # some common statistics on each column, note you can scroll to the right

In [ ]:
df.head() # first few rows of data (default 5)

In [ ]:
df.tail(10) # last few rows of data (we specify 10 here)

Any individual `Series` can be fetched out of the data frame using square brackets, and some of the same functions apply, as well as a few others.

In [ ]:
df['iso_code']

In [ ]:
df['iso_code'].describe()

In [ ]:
df['iso_code'].head()

In [ ]:
df['iso_code'].tail()

In [ ]:
df['iso_code'].value_counts()

## Exercise
Go back into those cells above, and edit to investigate some other Series, like 'location', 'date', 'new_cases'

'value_counts()' has different top numbers for `iso_code` vs `location` vs `date`. What does that mean?

---


# Converting date strings to date-type objects
The `Series` 'date' looks like dates to us humans, but right now it is actually just text. Examine this difference:

In [ ]:
df['date'].describe()

In [ ]:
pd.to_datetime(df['date'])

It will be important for our matplotlib timeseries plots for this Series to be dates instead of text. Instead of typing the extra 'pd.to_datetime' every time, we can modify this data for good:

In [ ]:
df['date'] = pd.to_datetime(df['date'])
df['date'].describe(datetime_is_numeric=True)

# Data Slicing/Filtering
A smaller `DataFrame` can be created from `Series` of your choosing. As always in Python, take careful note of the syntax.

In [ ]:
df2 = df[  ['iso_code', 'location', 'date', 'new_cases', 'new_deaths', 'total_cases', 'new_cases_smoothed_per_million', 'new_deaths_smoothed_per_million']  ]
df2.head()

A `DataFrame` can be 'sliced' (filtered) based on conditions applied to the data. These actions can be read something like "The new variable USA is a `DataFrame` made from df2 by selecting all the rows for which the 'iso_code' is equal to the text constant 'USA'"

**Critically important Python NOTE:** One = means the action **assignment**, whatever is on the right, put it into the left. It is the same as Snap's `Set <variable> to <value>` from the  yellow Variables tab.  Two == means the **question** *are these two things the same* (or in this context, *where* are these two the same?), and is equivalent to the Predicate = in Snap!, from the green Operators tab.

In [ ]:
USA = df2[ df2['iso_code'] == 'USA' ]
ITA = df2[ df2['iso_code'] == 'ITA' ]
SWE = df2[ df2['iso_code'] == 'SWE' ]
KOR = df2[ df2['iso_code'] == 'KOR' ]

In [ ]:
# Use this cell to try various looks at these sliced DataFrames, like USA.info() or SWE.tail(), etc.
USA.info()

# Plotting Pandas Series' with MATPLOTLIB
Now that we have a handle on manipulating csv data with pandas, we turn to the main point, which is to be able to visualize the data graphically. Here's a naive plot to start with. 


In [ ]:
USAx = USA['date']       # grab the date       Series out of the USA DataFrame into a variable called USAx
USAy = USA['new_deaths'] # grab the new_deaths Series out of the USA DataFrame into a variable called USAy
plt.figure(figsize=(16,8))
plt.plot(USAx, USAy)
plt.plot(ITA['date'], ITA['new_deaths']) # The same can be done inside the plot command
plt.plot(SWE['date'], SWE['new_deaths'])
plt.plot(KOR['date'], KOR['new_deaths'])
plt.legend(['USA', 'Italy', 'Sweden', 'South Korea'])
plt.show()

First off, those curves are obviously incomparable because of quite different country populations. We can convert to per-million by applying arithmetic to the whole series (requires knowledge of the country population)

In [ ]:
USA['new_deaths']

In [ ]:
USA['new_deaths'] / 330000000   # USA population is about 350M, so this is per capita

In [ ]:
USA['new_deaths'] / 330000000 * 1000000  # per capita is too small, so scale up to per million

In [ ]:
# Same as the above plot, except scaling per million (USA~331M, etc)
USAperM = USA['new_deaths']/331
plt.figure(figsize=(16,8))
plt.plot(USAx, USAperM)
plt.plot(ITA['date'], ITA['new_deaths']/60) # again, all this can be done inline
plt.plot(SWE['date'], SWE['new_deaths']/10)
plt.plot(KOR['date'], KOR['new_deaths']/51)
plt.legend(['USA', 'Italy', 'Sweden', 'South Korea'])
plt.show()

Next let's smooth out those weekly cycles with a 7-day average

In [ ]:
USAperMrollWeek = USAperM.rolling(window=7) # this tracks statistics on a rolling window of 7 rows (days)
USAperMweekAvg  = USAperMrollWeek.mean()    # this grabs the mean() (average), as opposed to min/max/med, etc.
plt.figure(figsize=(16,8))
plt.plot(USAx, USAperMweekAvg)
plt.plot(ITA['date'], ITA['new_deaths'].rolling(window=7).mean()/60)  # These inlines are starting to get 
plt.plot(SWE['date'], SWE['new_deaths'].rolling(window=7).mean()/10)  #    unmanageably complex
plt.plot(KOR['date'], KOR['new_deaths'].rolling(window=7).mean()/51)
plt.legend(['USA', 'Italy', 'Sweden', 'South Korea'])
#plt.show()

---
# Optional extra: Shifted Dates

Most complicated, we can see that these curves would be more comparable if they were date-shifted, to reflect the different times when the pandemic hit different countries. A common technique is to line them all up based on when they had a certain common minimum number of cases, say 10. We will filter on a condition again.

In [ ]:
# I could just type 100 in every line below, but this way if I want to experiment with a different value
# I can edit just 1 line, instead of having to edit a line for every country (especially as countries are added)
min_cases = 1000
USAsh = USA[ USA['total_cases'] >= min_cases ]  # 'sh' for shift
ITAsh = ITA[ ITA['total_cases'] >= min_cases ]
SWEsh = SWE[ SWE['total_cases'] >= min_cases ]
KORsh = KOR[ KOR['total_cases'] >= min_cases ]

In [ ]:
USAsh['date']

In [ ]:
USAsh['date'].min()

In [ ]:
ITAsh['date'].min()

In [ ]:
USAsh['date'].min() - ITAsh['date'].min()

Now we can see above that Italy reached 100 cases on Feb 24, 8 days before the US on Mar 3. (And that date objects can be subtracted!)

Just like we were able to simply multiply and divide the entire 'new_deaths' Series by constant numbers, we can modify the 

In [ ]:
USAt0 = USAsh['date'].min()
ITAt0 = ITAsh['date'].min()
SWEt0 = SWEsh['date'].min()
KORt0 = KORsh['date'].min()

USAsh['date'] - USAt0

In [ ]:
USAshX = USAsh['date'] - USAt0
USAshY = USAsh['new_deaths'].rolling(window=7).mean()/331
plt.figure(figsize=(16,8))
plt.plot(USAshX, USAshY)

Note that plot goes from 0 to 2.5e16. Even though the description of `USAsh['date'] - USAt0` above says 'days', matplotlib is interpreting it as milliseconds. We can fix this by forcing conversion to days.

In [ ]:
USAshX = (USAsh['date'] - USAt0).astype('timedelta64[D]')
USAshX

In [ ]:

plt.figure(figsize=(16,8))
plt.plot(USAshX, USAshY)
plt.plot((ITAsh['date']-ITAt0).astype('timedelta64[D]'), ITAsh['new_deaths'].rolling(window=7).mean()/60)  
plt.plot((SWEsh['date']-SWEt0).astype('timedelta64[D]'), SWEsh['new_deaths'].rolling(window=7).mean()/10) 
plt.plot((KORsh['date']-KORt0).astype('timedelta64[D]'), KORsh['new_deaths'].rolling(window=7).mean()/51)
plt.legend(['USA', 'Italy', 'Sweden', 'South Korea'])
plt.xlim( (0, 300) )
plt.show()